In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from datetime import datetime
import optuna
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from optuna.samplers import TPESampler
import sqlalchemy
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, accuracy_score, mean_squared_error,mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from xgboost import XGBClassifier
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import Ridge  # 使用 Ridge 回归代替逻辑回归
from sklearn.preprocessing import StandardScaler
import seaborn as sns

C:\Users\Acer\anaconda3\envs\transformer_git\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 支持中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号 '-' 显示问题

 ## 达人商品数据

In [3]:
source_dir = r'F:\Graduate materials\论文\代码\92个达人的数据\新'
df =  pd.DataFrame()
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if '播主详情_常规商品' in file:
            file_name = source_dir + '\\'+ file
            df1 = pd.read_excel(file_name).iloc[0:2,0:2]
            df2 =  pd.read_excel(file_name,header=3)
            id = df1.iloc[0,1]
            fans = df1.iloc[1,1]
            df2['抖音号'] = id
            df2['粉丝数'] = fans
            df2 = df2[:-1]
            df = pd.concat([df,df2], ignore_index=True)


In [4]:
df.to_csv('F:\Graduate materials\论文\代码\output.csv', index=False)

## 商品特征处理

In [5]:
df = pd.read_csv('F:\Graduate materials\论文\代码\output.csv')
df['抖音号'] = df['抖音号'].astype(str)
df = df.drop(columns=['商品链接','商品飞瓜详情页','商品图片链接','店铺链接','品牌链接'])

In [6]:
def split_range(value):
    if '-' in value:
        # 如果是标准范围，按 '-' 拆分
        return value.split('-')
    else:
        # 如果是单一值，返回相同的最小值和最大值
        return [value, value]
    
    
def split_feature(df,feature_name):
    df[[feature_name+'最小值',feature_name+'最大值']] = df[feature_name].apply(split_range).apply(pd.Series)
    df[feature_name+'最小值'] = pd.to_numeric(df[feature_name+'最小值'], errors='coerce')
    df[feature_name+'最大值'] = pd.to_numeric(df[feature_name+'最大值'], errors='coerce')
    
for feature in ['商品价格','销售均价']:
    split_feature(df,feature)


In [7]:
# 自定义解析值的函数，处理 'w' 和 '亿+' 单位
def parse_value(v):
    v = v.strip()  # 去除空格
    if 'w+' in v:
        return float(v.replace('w+', '')) * 10001
    if 'w' in v:
        return float(v.replace('w', '')) * 10000  # 将 'w' 替换并转换为数值
    if '亿+' in v:
        return float(v.replace('亿+', '')) * 100000001  # 将 '亿' 替换并转换为数值
    if '亿' in v:
        return float(v.replace('亿', '')) * 100000000
    if '%+' in v:
        return float(v.replace('%+', '')) * 0.011
    if '%' in v:
        return float(v.replace('%', '')) * 0.01
    return float(v)  # 其他情况直接转换为数值


# 自定义函数：将单一值 '0' 转换为 '0-0'
def normalize_range(value):
    if value == '0':
        return '0-0'  # 将单一值 '0' 变为 '0-0'
    return value  # 其他情况保持原样


# 自定义标签编码函数：按最大值排序并编码
def label_encode_sorted(df, column_name):
    # 先将单一值 '0' 转换为 '0-0'
    df[column_name] = df[column_name].apply(normalize_range)

    # 获取所有唯一的范围，并按最大值排序
    unique_values = sorted(
        df[column_name].unique(),
        key=lambda x: parse_value(split_range(x)[1])  # 按最大值排序
    )
    print(unique_values)
    # 创建范围到编码的映射
    mapping = {value: idx for idx, value in enumerate(unique_values)}

    # 添加编码列
    df[column_name + '_编码'] = df[column_name].map(mapping)
    


for feature in ['销售额','销量','视频销售额','视频销量','直播销售额','直播销量','转化率']:
    label_encode_sorted(df,feature)


['0-0', '0-250', '250-500', '500-750', '750-1000', '1000-2500', '2500-5000', '5000-7500', '7500-1w', '1w-2.5w', '2.5w-5w', '5w-7.5w', '7.5w-10w', '10w-25w', '25w-50w', '50w-75w', '75w-100w', '100w-250w', '250w-500w', '500w-750w', '750w-1000w', '1000w-2500w', '2500w-5000w', '5000w-7500w', '7500w-1亿', '1亿+']
['0-0', '0-50', '50-100', '100-250', '250-500', '500-750', '750-1000', '1000-2500', '2500-5000', '5000-7500', '7500-1w', '1w-2.5w', '2.5w-5w', '5w-7.5w', '7.5w-10w', '10w-25w', '25w-50w', '50w-75w', '75w-100w', '100w+']
['0-0', '0.0', '0-250', '250-500', '500-750', '750-1000', '1000-2500', '2500-5000', '5000-7500', '7500-1w', '1w-2.5w', '2.5w-5w', '5w-7.5w', '7.5w-10w', '10w-25w', '25w-50w', '50w-75w', '75w-100w', '100w-250w', '250w-500w', '500w-750w', '1000w-2500w']
['0-0', '0.0', '0-50', '50-100', '100-250', '250-500', '500-750', '750-1000', '1000-2500', '2500-5000', '5000-7500', '7500-1w', '1w-2.5w', '2.5w-5w', '10w-25w']
['0-0', '0-250', '250-500', '500-750', '750-1000', '1000-25

In [8]:
df['上架时间'] = pd.to_datetime(df['上架时间'], format='mixed', errors='coerce')
now = datetime.now()
df['上架距今(天)'] = (now - df['上架时间']).dt.days

In [9]:
df = df.drop(columns = ['商品价格','销售均价','销售额','销量','视频销售额','视频销量','直播销售额','直播销量','转化率','上架时间'])

## 达人视频数据

In [10]:
source_dir = r'F:\Graduate materials\论文\代码\92个达人的数据\新'
df_u =  pd.DataFrame()
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if '视频记录导出' in file:
            file_name = source_dir + '\\'+ file
            df1 = pd.read_excel(file_name).iloc[0:2,0:2]
            df2 =  pd.read_excel(file_name,header=3)
            df2['互动率'] =df2['互动率'].str.replace('%', '').astype(float) / 100
            for feature in df2.columns:
                if feature not in ['视频热度','点赞','评论','分享','收藏','互动量','互动率']:
                    df2 = df2.drop(columns = feature)
            df3 = df2.mean()
            id = df1.iloc[0,1]
            df3['抖音号'] = id
            df3['视频数'] = df2['互动率'].count()
            df_u = pd.concat([df_u,df3.to_frame().T ], ignore_index=True)
df_u

,视频热度,点赞,评论,分享,收藏,互动量,互动率,抖音号,视频数
0,6.1434,15.563333,1.383333,4.143333,4.083333,25.173333,0.002402,58527959436,300
1,12.223155,1010.037433,96.26738,253.074866,284.818182,1644.197861,0.005302,appleqcsm,187
2,15.9624,6869.46,803.52,747.84,680.13,9100.95,0.007247,mikebuke,100
3,8.688635,73.851406,14.261044,2.927711,6.064257,97.104418,0.004603,liguangai_,249
4,22.092647,5309.582353,567.194118,110.811765,151.8,6139.388235,0.036211,MRHALA,170
...,...,...,...,...,...,...,...,...,...
87,18.067233,26021.57,1060.946667,2546.98,2127.226667,31756.723333,0.006353,chensanfeigg31,300
88,9.438933,257.256667,9.28,13.23,16.42,296.186667,0.005335,love_Peaches11,300
89,12.14553,2127.159091,47.727273,70.121212,511.560606,2756.568182,0.017139,628027715,132
90,12.712941,2634.878431,242.470588,1487.592157,155.211765,4520.152941,0.004439,27948777,255


In [11]:
df_u['抖音号'] = df_u['抖音号'].astype(str)
df_merge = pd.merge(df,df_u,how='left',on='抖音号')

In [12]:
df_merge.to_csv('F:\Graduate materials\论文\代码\output_combined.csv', index=False)